In [1]:
import os
os.environ["HF_HOME"] = "E:/huggingface"

#import transformers
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


dataset = load_dataset("squad_it")
DEVICE = "cuda:0"

In [2]:
from get_model import get_model
model_name = "maestrale"

model, tokenizer = get_model(model_name)
#model = model.to(DEVICE) #togliere se in 4 bit

bin c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from get_prompt import get_prompt
# del generate_prompt
generate_prompt, stop = get_prompt(model_name) #stop sembra useless

In [4]:
def build_question(context, question):
    return f"Dato il seguente testo:\n{context}\nRispondi brevemente a questa domanda:\n{question}"

def build_answer(answer):
    return f"Risposta breve: {answer}"

In [5]:
import random

def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["context"], elem["question"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="assistant",
            text=build_answer(elem["answers"]["text"][0])
        ))
    return conversation

In [6]:
chat = [
  {"role": "system", "content": "Sei un assistente utile." },
  {"role": "user", "content": "Ciao come va amico?" },
  {"role": "assistant", "content": "Tutto bene grazie." },
  {"role": "user", "content": "E tua sorella Alice?" },
  {"role": "assistant", "content": "è morta"},
  {"role": "user", "content": "Ah cavolo, mi dispiace."},
  {"role": "assistant", "content": "tranqui bro, è la vita"},
  {"role": "user", "content": "scusa come si chiama tua sorella?"},
] 
print(tokenizer.apply_chat_template(chat, tokenize=False))

<|im_start|>system
Sei un assistente utile.<|im_end|>
<|im_start|>user
Ciao come va amico?<|im_end|>
<|im_start|>assistant
Tutto bene grazie.<|im_end|>
<|im_start|>user
E tua sorella Alice?<|im_end|>
<|im_start|>assistant
è morta<|im_end|>
<|im_start|>user
Ah cavolo, mi dispiace.<|im_end|>
<|im_start|>assistant
tranqui bro, è la vita<|im_end|>
<|im_start|>user
scusa come si chiama tua sorella?<|im_end|>



In [7]:
import torch

def forward_model(*, prompt, model, tokenizer):

    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            pad_token_id=tokenizer.eos_token_id,
            prompt_lookup_num_tokens=10,
            temperature = 0.7,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.2
        )
        #print(output_ids)

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        # print(generated_text)
        # print(generated_text[len(prompt[i]):])
        generated_text = generated_text[len(prompt[i]):] #perchè in output_ids c'è anche il prompt!

        # if "\n" in generated_text:
        #     generated_text = generated_text[:generated_text.index("\n")] non necessario
        
        ret.append(generated_text.strip())

    return ret

In [8]:
import time

t0 = time.time()
out = forward_model(
    prompt=["ciao amico come va?"],
    model=model,
    tokenizer=tokenizer,
)
print(time.time() - t0)

print(out)

11.321749210357666
['ti scrivo perché vorrei comprare un nuovo smartphone e ho già pensato a qualche modello.ho letto la t']


In [14]:
### CELLA DI PROVA DEL NUOVO GET PROMPT DI SAIGA-ITA-7B!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

total = 0
for i in range(3):
    elem     = ds[i].copy()
    ids      = elem["id"]
    context  = elem["context"]
    question = elem["question"]
    answers  = elem["answers"]

    model_inputs = []
    chat = get_shots(dataset, 3) + [
                dict(
                    role="user",
                    text=build_question(context, question)
                )
            ]
    model_inputs.append(
        generate_prompt(
        # scrive tre esempi non come  prompt ma come cronologia di chat: role: user -> text: domanda, role: assistant -> text: risposta breve
            chat,
            do_continue=True
        ) + "" + build_answer("").strip()
    )

print(model_inputs[0])

<|im_start|>system
Sei un assistente utile.<|im_end|>
<|im_start|>user
Dato il seguente testo:
La produzione di beni e servizi prodotti dal lavoro e dagli immobili situati negli Stati Uniti è diminuita a un tasso annuo di circa il 6% nel quarto trimestre del 2008 e nel primo trimestre del 2009, rispetto all' attività degli anni precedenti. Il tasso di disoccupazione negli Stati Uniti è salito al 10,1% nell' ottobre 2009, il tasso più elevato dal 1983 e circa il doppio di quello precedente alla crisi. Le ore medie per settimana lavorativa sono scese a 33, il livello più basso da quando il governo ha iniziato a raccogliere i dati nel 1964. Con il calo del prodotto interno lordo è venuto il declino dell' innovazione. Con meno risorse a rischio di distruzione creativa, il numero di domande di brevetto flat-lined. Rispetto agli ultimi 5 anni di crescita esponenziale delle domande di brevetto, questa stagnazione è correlata al calo analogo del PIL nello stesso periodo. 10.1%.
Rispondi brevem

In [15]:
import uuid
from tqdm import tqdm

batch_size = 2

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

bar = tqdm(ds.iter(batch_size=batch_size), total=len(ds))
total = 0
for elem in bar:
    try:
        ids = elem["id"]
        context = elem["context"]
        question = elem["question"]
        answers = elem["answers"]

        model_inputs = []
        for c, q in zip(context, question):
            model_inputs.append(
                generate_prompt(
                    get_shots(dataset, 3) + [
                        dict(
                            role="user",
                            text=build_question(c, q)
                        )
                    ],
                    do_continue=True
                ) + " " + build_answer("").strip()
            )
        # print(model_inputs[0])
        # break

        model_outputs = forward_model(
            prompt=model_inputs[0],
            model=model,
            tokenizer=tokenizer,
        )
        model_outputs = model_outputs[0].split("Risposta breve:")[-1] if "Risposta breve:" in model_outputs[0] else _
        #print(f"DOMANDA:\n{model_inputs[0]}\n\nRISPOSTA\n{model_outputs}\nSOLUZIONE\n{answers[0]['text']}\n")#[520:]

        # for model_output, id in zip(model_outputs, ids):
        #     predicted_answers.append(dict(
        #         id=id,
        #         prediction_text=model_output,
        #     ))
        predicted_answers.append(dict(
                id=ids[0],
                prediction_text=model_outputs,
            ))

        for ans, id in zip(answers, ids):
            theoretical_answers.append(dict(
                id=id,
                answers=ans,
            ))
        
    except Exception as e:
        print(e)
        continue

  4%|▍         | 329/7609 [34:40<10:01:07,  4.95s/it]

In [ ]:
import os
import json



# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)
with open(f"./cache/generated-squad-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)

In [ ]:
import json
with open(f"./cache/generated-squad-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

In [ ]:
import evaluate

predicted_answers = data["predicted_answers"]
theoretical_answers = data["theoretical_answers"]

metric = evaluate.load("squad")
results = metric.compute(predictions=predicted_answers, references=theoretical_answers)

import time


print("=== REPORT ===")
print("current date:", time.strftime("%d/%m/%Y %H:%M:%S"))
print("Dataset: SQuAD-it")
print("Model:", model_name)
print(results)
print("==========================")